In [ ]:
import dask.bag as db
from glob import glob
import pandas as pd
import re
import numpy as np

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objs as go

In [ ]:
'''
This code snippet and Data is courtesy of Thiago Sanches from Palmer Lab
'''

# allgwas = glob(f'''/tscc/projects/ps-palmer/gwas/projects/*/results/gwas/*_chrgwas*.mlma''') 
#### here in the second * you can specify a chromosome or a list of chromosomes this one will load everything

target_1 = pd.read_csv('target_1.txt', header = None, names=['SNP'])
target_1 = list(target_1.SNP)

allgwas = glob(f'''/tscc/projects/ps-palmer/gwas/projects/*/results/gwas/*_chrgwas1.mlma''') #### example for chr 5 
seq = db.from_sequence(allgwas[:])
gwas_reader = lambda x: pd.read_csv(x, dtype={'Chr':int, 'SNP':str, 'bp':int}, sep = '\t')\
                              .assign(trait = x.split('regressedlr_')[-1].split('_chrgwas')[0],
                                      project = re.findall('gwas/projects/(.*)/results', x)[0],
                                     )\
                              .set_index('SNP')
# .reindex(target_1)
#### you can subset in this reader function to speed up the code too, the index is the snp names, so if you have a list you can use that
 ### if you want to subset at this point
allgwas_res = pd.concat(seq.map(gwas_reader).compute())

In [ ]:
allgwas_res['-log10 P-Value'] = -np.log10(allgwas_res['p'])

# allgwas_res_sorted = allgwas_res.sort_values(by='-log10 P-Value', ascending=False)

# Drop duplicates based on the 'trait' column, keeping the first occurrence (largest p-value)
# allgwas_res_no_duplicates = allgwas_res_sorted.drop_duplicates(subset='trait', keep='first')

# Get the top 20 traits with the largest p-values (if you have more than 20)
# top_20_traits = allgwas_res_no_duplicates.nlargest(20, '-log10 P-Value')

# li = top_20_traits.trait

# phenotype = allgwas_res[allgwas_res.trait.isin(li)]

# phenotype.bp = phenotype.bp.astype(int)
# phenotype.Chr = phenotype.Chr.astype(int)
# append_position = phenotype.groupby('Chr').bp.agg('max').sort_index().cumsum().shift(1,fill_value=0)
# phenotype['Chromosome'] = phenotype.apply(lambda row: int(row.bp) + append_position[row.Chr], axis = 1)
# phenotype

In [ ]:
allgwas_res.bp = allgwas_res.bp.astype(int)
allgwas_res.Chr = allgwas_res.Chr.astype(int)
append_position = allgwas_res.groupby('Chr').bp.agg('max').sort_index().cumsum().shift(1,fill_value=0)
allgwas_res['Chromosome'] = allgwas_res.apply(lambda row: int(row.bp) + append_position[row.Chr], axis = 1)
allgwas_res

In [ ]:
#position the x-axis by substituting the distance with positions in bp

li = top_20_traits.trait

phenotype = allgwas_res[allgwas_res.trait.isin(li)]

phenotype.bp = phenotype.bp.astype(int)
phenotype.Chr = phenotype.Chr.astype(int)
append_position = phenotype.groupby('Chr').bp.agg('max').sort_index().cumsum().shift(1,fill_value=0)
phenotype['Chromosome'] = phenotype.apply(lambda row: int(row.bp) + append_position[row.Chr], axis = 1)

phenotype = phenotype.reset_index().rename(columns={'index': 'SNP'})

highlighted_snp = '1:78033225'

phenotype['highlight'] = phenotype.SNP.apply(lambda x: 'Highlighted' if x == highlighted_snp else 'Normal')

# Create a scatter plot with phenotypes on x and p-values on y
fig = px.scatter(
    phenotype, x='bp', y='-log10 P-Value',
    hover_name=phenotype.index,  # Show SNP names on hover
    title='Target SNP: 1:78033225 (3MB surround)',
    color = 'trait',
    size=phenotype['SNP'].apply(lambda x: 20 if x == highlighted_snp else 5),
    labels={'SNP': 'SNP', '-log10 P-Value': '-log10 P-Value'},  # Color points by phenotype
)

fig.update_traces(marker=dict(line=dict(width=0, color='DarkSlateGrey')))

fig.update_traces(
    selector=dict(mode='markers'),  # Ensure we are customizing marker properties
    marker=dict(
        line=dict(
            width=[4 if highlight == 'Highlighted' else 0 for highlight in phenotype['highlight']]
        )
    )
)

fig.update_layout(
    width=1000,  # Set the width of the plot
    height=800   # Set the height of the plot
)

# Show the interactive plot
fig.show()

In [ ]:
#Manhattan plot of Chromosome 1 for Trait: kidney_left_g

phenotype = allgwas_res[allgwas_res.trait == 'liver_left_weight']

phenotype = phenotype.reset_index().rename(columns={'index': 'SNP'})

highlighted_snp = '1:78033225'

highest_snp = phenotype.loc[phenotype['-log10 P-Value'].idxmax(), 'SNP']

phenotype['highlight'] = phenotype.SNP.apply(lambda x: 'Highlighted' if x == highlighted_snp else 'Normal')

# Create a scatter plot with phenotypes on x and p-values on y
fig = px.scatter(
    phenotype, x='bp', y='-log10 P-Value',
    hover_name=phenotype.index,  # Show SNP names on hover
    title='Chrom 1, Target SNP: 1:78033225, Trait: liver_left_weight',
    size=phenotype['SNP'].apply(lambda x: 10 if x == highlighted_snp else 2),  # Color points by phenotype
)

fig.update_traces(marker=dict(line=dict(width=0, color='DarkSlateGrey')))

fig.update_traces(
    selector=dict(mode='markers'),  # Ensure we are customizing marker properties
    marker=dict(
        line=dict(
            width=[4 if highlight == 'Highlighted' else 0 for highlight in phenotype['highlight']]
        )
    )
)

target_idx = phenotype[phenotype['SNP'] == highlighted_snp].index[0]  # First occurrence of the target SNP
fig.add_annotation(
    x=phenotype['bp'][target_idx],
    y=phenotype['-log10 P-Value'][target_idx],
    text=f"Target: {highlighted_snp}",
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=-40,
    bgcolor="yellow"
)

# Annotate the highest SNP
highest_idx = phenotype['-log10 P-Value'].idxmax()  # Index of the highest SNP
fig.add_annotation(
    x=phenotype['bp'][highest_idx],
    y=phenotype['-log10 P-Value'][highest_idx],
    text=f"Highest: {highest_snp}",
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=-40,
    bgcolor="lightgreen"
)

fig.update_layout(
    width=1500,  # Set the width of the plot
    height=800,   # Set the height of the plot
    showlegend=False
)

# Show the interactive plot
fig.write_image("fig5.png")